# 画像異常検知 実験ノート

複数バックボーン × 複数カテゴリの組み合わせで Mahalanobis 距離法と PaDiM を評価し、OOF スコアから求めた z 閾値で最終モデルを比較


## 1. ライブラリ読み込み

In [1]:
import os
import json
from datetime import datetime
from pathlib import Path
from dataclasses import dataclass
from typing import Any, Callable, Dict, List, Tuple

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset, Subset

from PIL import Image
from sklearn.covariance import ledoit_wolf
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.models import get_model, get_model_weights

import random
import warnings

# 独自モジュール
from anomaly_detectors import (
    fit_mahalanobis,
    all_mahalanobis_scores,
    fit_padim,
    all_padim_scores,
)


## 2. 共通設定と実験用ディレクトリ
乱数シードやデバイス、結果保存先などをまとめて初期化します。


In [2]:
# 共通設定をまとめて定義し、再現性と出力先を揃える
RANDOM_SEED = 0
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PIN_MEMORY = device.type == "cuda"
NUM_WORKERS = min(4, os.cpu_count() or 1)

MVTEC_ROOT = Path(os.environ.get("MVTEC_ROOT", "datasets/MVTecAD"))
MVTEC_ROOT.mkdir(parents=True, exist_ok=True)

RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M")
RESULT_ROOT = Path(f"results/dev/{RUN_TIMESTAMP}")
RESULT_ROOT.mkdir(parents=True, exist_ok=True)

DEFAULT_BATCH_SIZE = 16
DEFAULT_K_SPLITS = 5
threshold_percentile = 99
padim_channel_subsample = 100

# 実行対象のバックボーンとカテゴリ
TARGET_BACKBONES = ["resnet18", "efficientnet_b0"]
TARGET_CATEGORIES = ["carpet", "leather", "tile", "bottle", "cable", "pill"]

warnings.filterwarnings("ignore", category=UserWarning)
print(f"[INFO] device={device}, pin_memory={PIN_MEMORY}, num_workers={NUM_WORKERS}")
print(f"[INFO] result root: {RESULT_ROOT}")


[INFO] device=cpu, pin_memory=False, num_workers=4
[INFO] result root: results/dev/202509212215


### 2.2 データ取得（anomalib）
MVTec のデータが存在しない場合は anomalib を用いてダウンロードします。


In [3]:
# MVTec データの有無を確認し、必要なら anomalib でダウンロード
try:
    from anomalib.data import MVTecAD
except ImportError:
    print("[WARN] anomalib が見つかりません。事前にインストールしてください。")
else:
    try:
        datamodule = MVTecAD(root=str(MVTEC_ROOT))
        datamodule.prepare_data()
        print(f"[INFO] downloaded/prepared MVTec data under {MVTEC_ROOT}")
    except Exception as exc:
        print(f"[WARN] anomalib のデータ準備でエラー: {exc}")


[INFO] downloaded/prepared MVTec data under datasets/MVTecAD


## 3. バックボーン読み込みヘルパー
`torchvision` のモデル名からバックボーンを構築します。


In [4]:
def load_backbone_from_name(name: str) -> nn.Module:
    """バックボーン名から学習済みモデルを読み込み、評価モードで返す。"""
    try:
        weights = None
        try:
            weights = get_model_weights(name).DEFAULT
        except (AttributeError, ValueError, RuntimeError):
            pass  # ウェイトが無いモデルはランダム初期化
        model = get_model(name, weights=weights)
    except Exception as exc:
        raise ValueError(f"未対応のバックボーンです: {name}") from exc
    return model.eval().to(device)


## 4. データセット関連ユーティリティ
MVTec データの読み込み、KFold 分割、DataLoader の組み立てを関数化します。


In [5]:
def make_transform(image_size: int) -> transforms.Compose:
    """画像サイズに合わせた前処理パイプラインを作成する。"""
    imagenet_mean = [0.485, 0.456, 0.406]
    imagenet_std = [0.229, 0.224, 0.225]
    return transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
    ])


class ImagePathDataset(Dataset):
    """ファイルパスから画像テンソルとラベルを返す簡易データセット。"""

    def __init__(self, paths: List[Path], labels: List[Any], transform=None):
        self.paths = [Path(p) for p in paths]
        self.labels = labels
        self.transform = transform

    def __len__(self) -> int:
        return len(self.paths)

    def __getitem__(self, idx: int):
        img_path = self.paths[idx]
        label = self.labels[idx]
        with Image.open(img_path) as img:
            img = img.convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img, label


def _existing_category_root(category: str) -> Path:
    """カテゴリのルートディレクトリを既知のパターンから探す。"""
    candidates = [
        MVTEC_ROOT / category,
        Path("datasets/MVTecAD") / category,
        Path("MVtec_dataset") / category,
    ]
    for candidate in candidates:
        if candidate.exists():
            return candidate
    raise FileNotFoundError(f"MVTec category not found: {category}")


def _list_images(directory: Path) -> List[Path]:
    """指定ディレクトリ以下の画像パスを拡張子フィルタ付きで列挙する。"""
    exts = {".png", ".jpg", ".jpeg"}
    if not directory.exists():
        return []
    return sorted([p for p in directory.rglob('*') if p.suffix.lower() in exts])


def build_full_train_loader(
    category: str,
    *,
    batch_size: int,
    transform,
    pin_memory: bool,
    num_workers: int,
) -> DataLoader:
    """カテゴリ全体の train/good 画像を読み込み用 DataLoader にする。"""
    root = _existing_category_root(category)
    train_paths = _list_images(root / 'train' / 'good')
    assert train_paths, f"No train/good images found for {category}"
    dataset = ImagePathDataset(train_paths, [0] * len(train_paths), transform=transform)
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin_memory,
    )


def build_cv_and_test_loaders(
    category: str,
    *,
    k_splits: int,
    batch_size: int,
    transform,
    pin_memory: bool,
    num_workers: int,
) -> Tuple[List[Dict[str, Any]], DataLoader]:
    """train/good を KFold で分割し、CV 用と dev test 用 DataLoader を返す。"""
    root = _existing_category_root(category)
    train_good = _list_images(root / 'train' / 'good')
    assert train_good, f"No train/good images found for {category}"

    base_ds = ImagePathDataset(train_good, [0] * len(train_good), transform=transform)
    kf = KFold(n_splits=k_splits, shuffle=True, random_state=0)

    cv_folds: List[Dict[str, Any]] = []
    for fold_id, (tr_idx, va_idx) in enumerate(kf.split(range(len(train_good)))):
        train_subset = Subset(base_ds, tr_idx)
        val_subset = Subset(base_ds, va_idx)
        train_loader = DataLoader(
            train_subset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=num_workers,
            pin_memory=pin_memory,
        )
        val_loader = DataLoader(
            val_subset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=num_workers,
            pin_memory=pin_memory,
        )
        cv_folds.append(
            {
                "fold": fold_id,
                "train_loader": train_loader,
                "val_loader": val_loader,
                "n_train": len(tr_idx),
                "n_val": len(va_idx),
            }
        )

    test_dir = root / 'test'
    test_paths: List[Path] = []
    test_labels: List[str] = []
    if test_dir.exists():
        for sub in sorted([d for d in test_dir.iterdir() if d.is_dir()], key=lambda p: p.name):
            paths = _list_images(sub)
            if paths:
                test_paths.extend(paths)
                test_labels.extend([sub.name] * len(paths))
    assert test_paths, f"No test images found for {category}"

    test_dataset = ImagePathDataset(test_paths, test_labels, transform=transform)
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin_memory,
    )

    return cv_folds, test_loader


## 5. 可視化ユーティリティ
fold ごとのスコア分布をヒストグラムで保存します。


In [6]:
def plot_histogram(
    df: pd.DataFrame,
    ifold: int,
    save_dir: Path,
    method_name: str,
    show_image: bool = False,
) -> None:
    """スコアのヒストグラムを描画・保存し、必要に応じて表示する。"""
    df = df.copy()
    df["log_score"] = np.log1p(df["score"])

    fig, ax = plt.subplots(figsize=(8, 5))
    n_bins = 30
    bin_edges = np.linspace(df["log_score"].min(), df["log_score"].max(), n_bins + 1)

    preferred = ["train", "val", "good"]
    labels_all = list(dict.fromkeys(df["label"].tolist()))
    ordered_labels = preferred + [lb for lb in labels_all if lb not in preferred]

    for label in ordered_labels:
        values = df.loc[df["label"] == label, "log_score"].to_numpy()
        if len(values) == 0:
            continue
        ax.hist(
            values,
            bins=bin_edges,
            alpha=0.6,
            label=label,
            edgecolor="black",
            linewidth=0.3,
        )

    ax.set_title(f"{method_name} / Fold {ifold} — log(score)")
    ax.set_xlabel("log_score")
    ax.set_ylabel("count")
    ax.legend(fontsize=8, ncol=2)
    fig.tight_layout()

    save_dir.mkdir(parents=True, exist_ok=True)
    out_png = save_dir / f"fold_{ifold}_hist.png"
    fig.savefig(out_png, dpi=150)
    if show_image:
        plt.show()
    plt.close(fig)


## 6. パイプライン本体
Mahalanobis / PaDiM の学習・評価を一貫処理する関数を定義します。


In [7]:
def _to_cpu(obj):
    """テンソルやテンソルのリストを CPU 側に集約する補助関数。"""
    if isinstance(obj, torch.Tensor):
        return obj.detach().cpu()
    if isinstance(obj, list):
        return [_to_cpu(o) for o in obj]
    if isinstance(obj, tuple):
        return tuple(_to_cpu(o) for o in obj)
    return obj


@dataclass
class MethodSpec:
    key: str
    display_name: str
    fit_fn: Callable[..., Any]
    score_fn: Callable[..., Any]
    fit_kwargs: Dict[str, Any]
    score_kwargs_common: Dict[str, Any]
    score_kwargs_test: Dict[str, Any]
    output_dir: Path
    collects_heatmaps: bool = False


def save_padim_heatmaps(heatmaps, out_dir: Path, cmap: str = "hot") -> int:
    """PaDiM のヒートマップを保存し、保存枚数を返す。"""
    if heatmaps is None:
        return 0

    arrays = []
    for hm in heatmaps:
        if isinstance(hm, torch.Tensor):
            arrays.append(hm.detach().cpu().numpy())
        else:
            arrays.append(np.asarray(hm))

    if not arrays:
        return 0

    vmin = float(min(hm.min() for hm in arrays))
    vmax = float(max(hm.max() for hm in arrays))
    out_dir.mkdir(parents=True, exist_ok=True)
    for idx, hm in enumerate(arrays):
        out_png = out_dir / f"heatmap_{idx:05d}.png"
        plt.imsave(out_png, hm, cmap=cmap, vmin=vmin, vmax=vmax)
    return len(arrays)


def run_pipeline(
    spec: MethodSpec,
    model: nn.Module,
    *,
    category: str,
    transform,
    device: torch.device,
    k_splits: int,
    batch_size: int,
    oof_percentile: float,
    pin_memory: bool,
    num_workers: int,
    plot_histogram_fn: Callable[..., None] = plot_histogram,
) -> Dict[str, Any]:
    """1 メソッド分の CV→OOF→最終モデル学習を実行する。"""
    spec.output_dir.mkdir(parents=True, exist_ok=True)

    cv_folds, dev_test_loader = build_cv_and_test_loaders(
        category=category,
        k_splits=k_splits,
        batch_size=batch_size,
        transform=transform,
        pin_memory=pin_memory,
        num_workers=num_workers,
    )

    fold_artifacts: Dict[int, Dict[str, Any]] = {}
    metric_rows: List[Dict[str, float]] = []
    oof_normal_scores_per_fold: List[np.ndarray] = []

    test_labels = np.array([0 if lbl == "good" else 1 for lbl in dev_test_loader.dataset.labels])

    for ifold, fold in enumerate(cv_folds):
        fit_kwargs = dict(spec.fit_kwargs)
        fit_kwargs.setdefault("device", device)
        model_state = spec.fit_fn(fold["train_loader"], model, **fit_kwargs)

        scores_train = _to_cpu(spec.score_fn(model_state, fold["train_loader"], **spec.score_kwargs_common))
        scores_val = _to_cpu(spec.score_fn(model_state, fold["val_loader"], **spec.score_kwargs_common))
        oof_normal_scores_per_fold.append(scores_val.numpy())

        test_out = spec.score_fn(model_state, dev_test_loader, **spec.score_kwargs_test)
        if spec.collects_heatmaps:
            scores_test, heatmaps_test = test_out
        else:
            scores_test, heatmaps_test = test_out, None
        scores_test = _to_cpu(scores_test)
        if heatmaps_test is not None:
            heatmaps_test = _to_cpu(heatmaps_test)

        fold_artifacts[ifold] = {
            "model_state": model_state,
            "scores_train": scores_train,
            "scores_val": scores_val,
            "scores_test": scores_test,
            "heatmaps_test": heatmaps_test,
        }

        scores_val_np = scores_val.numpy()
        scores_test_np = scores_test.numpy()
        fold_threshold = float(np.quantile(scores_val_np, oof_percentile))
        preds_test = (scores_test_np >= fold_threshold).astype(int)
        auc = roc_auc_score(test_labels, scores_test_np)
        f1 = f1_score(test_labels, preds_test)
        metric_rows.append(
            {
                "fold": ifold,
                "threshold_raw@fold": fold_threshold,
                "auc": auc,
                "f1": f1,
            }
        )

        df_hist = pd.DataFrame(
            {
                "score": np.r_[
                    scores_train.numpy(),
                    scores_val.numpy(),
                    scores_test.numpy(),
                ],
                "label": (
                    ["train"] * len(scores_train)
                    + ["val"] * len(scores_val)
                    + dev_test_loader.dataset.labels
                ),
            }
        )
        plot_histogram_fn(df_hist, ifold, spec.output_dir, spec.display_name)

        print(
            f"[{spec.display_name} Fold {ifold}] Raw-th@{int(oof_percentile*100)}%: {fold_threshold:.4f}, "
            f"AUC: {auc:.4f}, F1: {f1:.4f}"
        )

    metrics_df = pd.DataFrame(metric_rows)
    metrics_df.to_csv(spec.output_dir / "summary.csv", index=False)
    auc_mean = float(metrics_df["auc"].mean())
    auc_std = float(metrics_df["auc"].std())
    f1_mean = float(metrics_df["f1"].mean())
    f1_std = float(metrics_df["f1"].std())
    print(
        f"[{spec.display_name}] CV AUC: {auc_mean:.4f} ± {auc_std:.4f}, "
        f"F1: {f1_mean:.4f} ± {f1_std:.4f}"
    )

    assert oof_normal_scores_per_fold, "OOF scores are empty."
    oof_normal_scores = np.concatenate(oof_normal_scores_per_fold)
    threshold_oof_raw = float(np.quantile(oof_normal_scores, oof_percentile))
    mu_oof = float(np.mean(oof_normal_scores))
    sigma_oof = float(np.std(oof_normal_scores, ddof=1))
    denom_oof = sigma_oof if sigma_oof > 0 else 1.0
    z_threshold = float(np.quantile((oof_normal_scores - mu_oof) / denom_oof, oof_percentile))

    print({
        "OOF_N": len(oof_normal_scores),
        "threshold_oof_raw": threshold_oof_raw,
        "mu_oof": mu_oof,
        "sigma_oof": sigma_oof,
        "z_threshold": z_threshold,
    })

    final_train_loader = build_full_train_loader(
        category,
        batch_size=batch_size,
        transform=transform,
        pin_memory=pin_memory,
        num_workers=num_workers,
    )

    final_fit_kwargs = dict(spec.fit_kwargs)
    final_fit_kwargs.setdefault("device", device)
    final_model_state = spec.fit_fn(final_train_loader, model, **final_fit_kwargs)

    final_train_scores = _to_cpu(
        spec.score_fn(final_model_state, final_train_loader, **spec.score_kwargs_common)
    ).numpy()
    mu_final = float(np.mean(final_train_scores))
    sigma_final = float(np.std(final_train_scores, ddof=1))
    denom_final = sigma_final if sigma_final > 0 else 1.0

    final_out = spec.score_fn(final_model_state, dev_test_loader, **spec.score_kwargs_test)
    if spec.collects_heatmaps:
        final_test_scores, final_heatmaps = final_out
    else:
        final_test_scores, final_heatmaps = final_out, None
    final_test_scores = _to_cpu(final_test_scores).numpy()
    final_preds = ((final_test_scores - mu_final) / denom_final > z_threshold).astype(int)
    final_auc = float(roc_auc_score(test_labels, final_test_scores))
    final_f1 = float(f1_score(test_labels, final_preds))

    print(
        f"[{spec.display_name} Final] AUC: {final_auc:.4f}, F1(z@{int(oof_percentile*100)}%): {final_f1:.4f}"
    )

    return {
        "fold_artifacts": fold_artifacts,
        "metrics": metrics_df,
        "auc_mean": auc_mean,
        "auc_std": auc_std,
        "f1_mean": f1_mean,
        "f1_std": f1_std,
        "oof": {
            "oof_normal_scores_per_fold": oof_normal_scores_per_fold,
            "threshold_oof_raw": threshold_oof_raw,
            "mu_oof": mu_oof,
            "sigma_oof": sigma_oof,
            "z_threshold": z_threshold,
        },
        "final": {
            "model_state": final_model_state,
            "mu_final": mu_final,
            "sigma_final": sigma_final,
            "test_scores": final_test_scores,
            "preds": final_preds,
            "auc": final_auc,
            "f1": final_f1,
            "heatmaps_test": final_heatmaps,
        },
    }


## 7. スイープ設定と実行
バックボーン・カテゴリの組み合わせでパイプラインをまとめて実行します。


In [8]:
BACKBONE_CFG = {
    "resnet18": {
        "image_size": 256,
        "md_layer": "flatten",
        "padim_layers": ["layer1.1.relu_1", "layer2.1.relu_1", "layer3.1.relu_1"],
        "d": padim_channel_subsample,
    },
    "efficientnet_b0": {
        "image_size": 256,
        "md_layer": "flatten",
        "padim_layers": [
            "features.6.3.add",
            "features.7.0.block.0",
            "features.7.0.block.1",
            "features.7.0.block.2",
            "features.7.0.block.3",
        ],
        "d": padim_channel_subsample,
    },
}

sweep_records: List[Dict[str, Any]] = []

for backbone_name in TARGET_BACKBONES:
    if backbone_name not in BACKBONE_CFG:
        print(f"[WARN] 未対応バックボーンをスキップ: {backbone_name}")
        continue

    cfg = BACKBONE_CFG[backbone_name]
    transform = make_transform(cfg["image_size"])
    model = load_backbone_from_name(backbone_name)

    for category in TARGET_CATEGORIES:
        combo_root = RESULT_ROOT / backbone_name / category
        md_dir = combo_root / "MD"
        padim_dir = combo_root / "PaDiM"
        md_dir.mkdir(parents=True, exist_ok=True)
        padim_dir.mkdir(parents=True, exist_ok=True)

        method_specs = [
            MethodSpec(
                key="mahalanobis",
                display_name="Mahalanobis",
                fit_fn=fit_mahalanobis,
                score_fn=all_mahalanobis_scores,
                fit_kwargs={"feature_node": cfg["md_layer"]},
                score_kwargs_common={},
                score_kwargs_test={},
                output_dir=md_dir,
            ),
            MethodSpec(
                key="padim",
                display_name="PaDiM",
                fit_fn=fit_padim,
                score_fn=all_padim_scores,
                fit_kwargs={"layers": cfg["padim_layers"], "d": cfg["d"]},
                score_kwargs_common={},
                score_kwargs_test={"return_maps": True},
                output_dir=padim_dir,
                collects_heatmaps=True,
            ),
        ]

        for spec in method_specs:
            print(f"[RUN] backbone={backbone_name}, category={category}, method={spec.display_name}")
            result = run_pipeline(
                spec=spec,
                model=model,
                category=category,
                transform=transform,
                device=device,
                k_splits=DEFAULT_K_SPLITS,
                batch_size=DEFAULT_BATCH_SIZE,
                oof_percentile=threshold_percentile / 100.0,
                pin_memory=PIN_MEMORY,
                num_workers=NUM_WORKERS,
            )

            sweep_records.append(
                {
                    "backbone": backbone_name,
                    "category": category,
                    "method": spec.display_name,
                    "cv_auc_mean": result["auc_mean"],
                    "cv_auc_std": result["auc_std"],
                    "cv_f1_mean": result["f1_mean"],
                    "cv_f1_std": result["f1_std"],
                    "final_auc": result["final"]["auc"],
                    "final_f1": result["final"]["f1"],
                    "z_threshold": result["oof"]["z_threshold"],
                    "mu_final": result["final"]["mu_final"],
                    "sigma_final": result["final"]["sigma_final"],
                }
            )

            if spec.key == "padim":
                saved = save_padim_heatmaps(
                    result["final"]["heatmaps_test"],
                    spec.output_dir / "final_heatmaps",
                )
                if saved:
                    print(f"[INFO] Saved {saved} heatmaps -> {spec.output_dir / 'final_heatmaps'}")

sweep_df = pd.DataFrame(sweep_records)
sweep_csv = RESULT_ROOT / "sweep_summary.csv"
sweep_df.to_csv(sweep_csv, index=False)
print(f"[INFO] Saved sweep summary CSV: {sweep_csv}")


[RUN] backbone=resnet18, category=carpet, method=Mahalanobis
[Mahalanobis Fold 0] Raw-th@99%: 27.4668, AUC: 0.8355, F1: 0.8343
[Mahalanobis Fold 1] Raw-th@99%: 27.2209, AUC: 0.8355, F1: 0.8343
[Mahalanobis Fold 2] Raw-th@99%: 31.9090, AUC: 0.8391, F1: 0.8024
[Mahalanobis Fold 3] Raw-th@99%: 31.7203, AUC: 0.8379, F1: 0.8024
[Mahalanobis Fold 4] Raw-th@99%: 28.4794, AUC: 0.8315, F1: 0.8324
[Mahalanobis] CV AUC: 0.8359 ± 0.0029, F1: 0.8211 ± 0.0171
{'OOF_N': 280, 'threshold_oof_raw': 29.826434440612786, 'mu_oof': 22.00357437133789, 'sigma_oof': 2.471437454223633, 'z_threshold': 3.1653077149391144}
[Mahalanobis Final] AUC: 0.8399, F1(z@99%): 0.8641
[RUN] backbone=resnet18, category=carpet, method=PaDiM
[PaDiM Fold 0] Raw-th@99%: 80.2876, AUC: 0.9430, F1: 0.9432
[PaDiM Fold 1] Raw-th@99%: 90.3524, AUC: 0.9506, F1: 0.9070
[PaDiM Fold 2] Raw-th@99%: 113.3250, AUC: 0.9494, F1: 0.8312
[PaDiM Fold 3] Raw-th@99%: 68.8588, AUC: 0.9430, F1: 0.9399
[PaDiM Fold 4] Raw-th@99%: 73.3449, AUC: 0.9539, F1

## 8. スイープ結果プレビュー
集計結果を DataFrame として確認します。


In [9]:
if not sweep_records:
    print("[WARN] まだスイープが実行されていません。前セルを実行してください。")
else:
    display(sweep_df)


,backbone,category,method,cv_auc_mean,cv_auc_std,cv_f1_mean,cv_f1_std,final_auc,final_f1,z_threshold,mu_final,sigma_final
0,resnet18,carpet,Mahalanobis,0.835875,0.002921,0.821146,0.017135,0.839888,0.864078,3.165308,13.384072,0.527227
1,resnet18,carpet,PaDiM,0.947994,0.004820,0.914408,0.049465,0.947030,0.902564,3.371017,32.315998,5.453840
2,resnet18,leather,Mahalanobis,0.964606,0.001688,0.919343,0.007276,0.968071,0.851852,3.299660,12.871729,0.500292
3,resnet18,leather,PaDiM,0.995720,0.001262,0.969347,0.011897,0.996264,0.929293,4.033716,30.507725,4.622657
4,resnet18,tile,Mahalanobis,0.983550,0.000975,0.937432,0.028033,0.984127,0.835821,2.564483,13.302982,0.473344
5,resnet18,tile,PaDiM,0.921284,0.005853,0.292793,0.211091,0.916306,0.878981,3.734488,34.350983,9.699724
6,resnet18,bottle,Mahalanobis,0.997302,0.000435,0.979220,0.004604,0.997619,0.863014,3.885338,11.811812,0.606500
7,resnet18,bottle,PaDiM,0.997302,0.002070,0.967492,0.012100,0.998413,0.984375,3.670456,78.146454,26.721533
8,resnet18,cable,Mahalanobis,0.922976,0.006603,0.747591,0.054031,0.931034,0.760331,3.042920,13.197304,0.322997
9,resnet18,cable,PaDiM,0.779423,0.014356,0.399929,0.147629,0.790667,0.614379,2.894491,51.414360,15.051565
